The Boston Housing Dataset

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of [ Boston MA](http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). The following describes the dataset columns:

* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per \$10,000
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in \$1000's



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from matplotlib import pyplot as plt

import seaborn as sns

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.
from pandas import read_csv
#Lets load the dataset and sample some
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
# Importation du dataset
df = read_csv('../input/housing.csv', header=None, delimiter=r"\s+", names=column_names)
print(df.head(5))

In [ ]:
df.info()

Il n'y a pas de valeur non-numérique, on peut donc continuer.  
On vérifie si des valeurs sont manquantes (NaN):

In [ ]:
df.count()

Toutes les colonnes sont à 506, il ne manque donc aucune valeur.

# Recherche de corrélations

On va utiliser la fonction "corr" pour calculer systématiquement le degré de corrélation entre deux paramètres :

In [ ]:
tabcorr = df.corr()

Utilisation d'une "heatmap" pour visualiser l'ensemble des corrélations :

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

Regroupement des paramètres par cluster classé par proximité:

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm")

On remarque 3 clusters principaux:
- RM, LSTAT et MEDV
- CRIL, RAD et TAX
- NOX, INDUS, DIS et AGE

On peu visiualiser différement ces cluster à l'aide d'un dendrogramme :

In [ ]:
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)

**On va s'intéresser maintenant et pour la suite** à la valeur médiane des logements (MEDV):

In [ ]:
correlations = tabcorr.MEDV
# on retire la ligne MEDV qui sera forcement à 1.00
correlations = correlations.drop(['MEDV'],axis=0)
# Affichage dans l'ordre décroissant ET on met tout en absolue car une valeur négative est aussi significative
print(abs(correlations).sort_values(ascending=False))

On remarque que ce qui a le plus corrélé sont LSTAT (% lower status of the population) et RM (average number of rooms per dwelling) avec ~69-74% comme ce qui avait pu être constaté avec le regroupement par cluster.

# Méthode #1: Régression linéaire multiple

Il faut dissocier les caractèristiques discrètes du reste, mais ici il n'y en a pas donc cela n'est pas nécessaire.

In [ ]:

#On crée la cible y (colonne 'MEVD')
y = df['MEDV']
#On crée les caractéristiques X
X = df.drop(['MEDV'], axis=1)

#Importation de la méthode
from sklearn.model_selection import train_test_split
#On sépare les ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

On utilise la fonction de régression linéaire multiple de sklearn :

In [ ]:
# Importation de la méthode
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
# apprentissage
lm.fit(X_train, y_train)
# prédiction
y_pred = lm.predict(X_test)

On trace un nuage de points pour comparer la prédiction et les résultats attendus :

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
# Distribution de l'erreur
sns.distplot(y_test-y_pred)

On remarque que le résultat possède une erreur entre -10 et +10 comparé à la valeur réelle du MEDV.  
  
On peut calculer une erreur moyenne à l'aide des moindres carrés :

In [ ]:
# Importation de la méthode 
from sklearn.metrics import mean_squared_error, r2_score
# Calcul
print(np.sqrt(mean_squared_error(y_test, y_pred)))

Sur les prédictions il y a une erreur moyenne de ~ 4.53 .
On va maintenant déterminer le score de précision R2 :

In [ ]:
from sklearn.metrics import r2_score
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

La prédiction possède une précision de 77.86% .

# Méthode #2: Régression par forêts aléatoires

Nous allons essayer d'obtenir un meilleur score à l'aide de la méthode cette méthode. Pour cela nous allons utiliser les forêts aléatoires pour la régression.

In [ ]:
#Importation de la méthode
from sklearn import ensemble
#Entrainement
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
# Prédiction
y_rf = rf.predict(X_test)
#Calcul du score
print(rf.score(X_test,y_test))
# Importation de la méthode 
from sklearn.metrics import mean_squared_error, r2_score
# Calcul
print(np.sqrt(mean_squared_error(y_test, y_rf)))

On obtient désormais un meilleur score (aléatoire mais en moyenne supérieur) avec une précision de moyenne 90% et un erreur moyenne comprise entre 2.4 et 3.2 pour la valeur de MEDV.  
  
On trace un nuage de points pour comparer la prédiction et les résultats attendus :

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

Ce nuage de points montre une plus petite disparité des points contrairement à la méthode précédente.

**Conclusion:** Cette méthode est bien meilleur que la précédente.

# Méthode #3: Gradient Boosting Regressor

Nous allons essayer d'obtenir un meilleur score à l'aide de la méthode cette méthode. 

In [ ]:
#  Importation de la méthode
from sklearn.ensemble import GradientBoostingRegressor
# Entrainement
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
# Prédiction
y_reg = reg.predict(X_test)
# Calcul du score
print(reg.score(X_test,y_test))
# Calcul de l'erreur moyenne
print(np.sqrt(mean_squared_error(y_test, y_reg)))

On remarque une nouvelle amélioration comparé aux méthodes précédentes avec un score de précision de 93.21% et une erreur moyenne de 2.5 pour la valeur de MEDV.

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_reg)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

**Conclusion:** Cette méthode est bien meilleur que les 2 précédentes comme l'on peut le voir avec le score de précision, l'erreur moyenne et la disparité des points.